#  Imports

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sb
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.encoding import OneHotEncoder as Ohe

from tensorflow import keras
from keras.callbacks import TensorBoard
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix, classification_report

#  Lecture du Dataset

In [4]:
import csv
import pandas as pd

# Chemin vers votre fichier CSV
fichier_csv = "Datasetnormal.csv"

# Ouvrir le fichier CSV en mode lecture avec l'encodage UTF-16
with open(fichier_csv, mode='r') as fichier:
    # Créer un lecteur CSV
    lecteur_csv = csv.DictReader(fichier)
    
    # Lire le fichier ligne par ligne et stocker dans une liste de dictionnaires
    donnees = [ligne for ligne in lecteur_csv if ligne]
colonnes = ['frame.number','ip.src','ip.dst','tcp.srcport','tcp.dstport','frame.len','ip.proto','tcp.flags.syn','tcp.flags.ack','tcp.flags.fin','ip.len','tcp.window_size_value','tcp.analysis.bytes_in_flight','tcp.analysis.push_bytes_sent','frame.time_epoch']
df = pd.DataFrame(donnees, columns=colonnes)
df['traficnature']='normal'
# Remplacer les chaînes de caractères vides par 0
df.replace("", 0, inplace=True)
print(df.head())

  frame.number      ip.src         ip.dst tcp.srcport tcp.dstport frame.len  \
0            1  10.0.0.134       10.0.0.2           0           0        87   
1            2  10.0.0.134       10.0.0.2           0           0        87   
2            3    10.0.0.2     10.0.0.134           0           0       151   
3            4    10.0.0.2     10.0.0.134           0           0       168   
4            5  10.0.0.134  18.154.41.124       41982         443        74   

  ip.proto tcp.flags.syn tcp.flags.ack tcp.flags.fin ip.len  \
0       17             0             0             0     73   
1       17             0             0             0     73   
2       17             0             0             0    137   
3       17             0             0             0    154   
4        6          True         False         False     60   

  tcp.window_size_value tcp.analysis.bytes_in_flight  \
0                     0                            0   
1                     0           

#  Vérification de la présence de valeurs Nan ou nulles et leur supression


In [5]:
number_null_value=df.isnull().sum()
number_null_value

frame.number                    0
ip.src                          0
ip.dst                          0
tcp.srcport                     0
tcp.dstport                     0
frame.len                       0
ip.proto                        0
tcp.flags.syn                   0
tcp.flags.ack                   0
tcp.flags.fin                   0
ip.len                          0
tcp.window_size_value           0
tcp.analysis.bytes_in_flight    0
tcp.analysis.push_bytes_sent    0
frame.time_epoch                0
traficnature                    0
dtype: int64

In [6]:
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]
df

,frame.number,ip.src,ip.dst,tcp.srcport,tcp.dstport,frame.len,ip.proto,tcp.flags.syn,tcp.flags.ack,tcp.flags.fin,ip.len,tcp.window_size_value,tcp.analysis.bytes_in_flight,tcp.analysis.push_bytes_sent,frame.time_epoch,traficnature
0,1,10.0.0.134,10.0.0.2,0,0,87,17,0,0,0,73,0,0,0,1722048111.463435359,normal
1,2,10.0.0.134,10.0.0.2,0,0,87,17,0,0,0,73,0,0,0,1722048111.463669322,normal
2,3,10.0.0.2,10.0.0.134,0,0,151,17,0,0,0,137,0,0,0,1722048111.475201161,normal
3,4,10.0.0.2,10.0.0.134,0,0,168,17,0,0,0,154,0,0,0,1722048111.507256173,normal
4,5,10.0.0.134,18.154.41.124,41982,443,74,6,True,False,False,60,64240,0,0,1722048111.508517278,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75470,75471,10.0.0.134,142.250.201.46,0,0,74,17,0,0,0,60,0,0,0,1722048714.254168297,normal
75471,75472,142.250.201.46,10.0.0.134,0,0,121,17,0,0,0,107,0,0,0,1722048714.264081591,normal
75472,75473,142.250.201.46,10.0.0.134,0,0,68,17,0,0,0,54,0,0,0,1722048714.264082128,normal
75473,75474,10.0.0.134,142.250.201.46,0,0,82,17,0,0,0,68,0,0,0,1722048714.264719748,normal


# Description de notre dataset

In [7]:
df.describe()

,frame.number,ip.src,ip.dst,tcp.srcport,tcp.dstport,frame.len,ip.proto,tcp.flags.syn,tcp.flags.ack,tcp.flags.fin,ip.len,tcp.window_size_value,tcp.analysis.bytes_in_flight,tcp.analysis.push_bytes_sent,frame.time_epoch,traficnature
count,75475,75475,75475,75475,75475,75475,75475,75475,75475,75475,75475,75475,75475,75475,75475,75475
unique,75475,73,74,186,186,1628,2,3,3,3,1630,80,1195,1078,75475,1
top,1,41.110.49.17,10.0.0.134,0,0,1399,17,0,0,0,1385,0,0,0,1722048111.463435359,normal
freq,1,46147,65919,64919,64919,57141,64919,64919,64919,64919,57141,64958,71066,71066,1,75475


In [8]:
columnsdrop=["ip.src","ip.dst",'frame.number','frame.time_epoch']
df=df.drop(columns=columnsdrop,axis=1)

In [9]:
df

,tcp.srcport,tcp.dstport,frame.len,ip.proto,tcp.flags.syn,tcp.flags.ack,tcp.flags.fin,ip.len,tcp.window_size_value,tcp.analysis.bytes_in_flight,tcp.analysis.push_bytes_sent,traficnature
0,0,0,87,17,0,0,0,73,0,0,0,normal
1,0,0,87,17,0,0,0,73,0,0,0,normal
2,0,0,151,17,0,0,0,137,0,0,0,normal
3,0,0,168,17,0,0,0,154,0,0,0,normal
4,41982,443,74,6,True,False,False,60,64240,0,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...
75470,0,0,74,17,0,0,0,60,0,0,0,normal
75471,0,0,121,17,0,0,0,107,0,0,0,normal
75472,0,0,68,17,0,0,0,54,0,0,0,normal
75473,0,0,82,17,0,0,0,68,0,0,0,normal


In [10]:
import re

# Fonction pour vérifier si une valeur est une adresse IP
def is_ip(value):
    # Expression régulière pour une adresse IP
    pattern = r"\b(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\b"
    return bool(re.match(pattern, str(value)))

# Appliquer la fonction aux colonnes tcp.srcport et tcp.dstport
df = df[~df['tcp.srcport'].apply(is_ip)]
df = df[~df['tcp.dstport'].apply(is_ip)]


In [11]:
# Convertir les valeurs booléennes en entiers
df['tcp.flags.syn'] = (df['tcp.flags.syn'] == 'True').astype(int)
df['tcp.flags.ack'] = (df['tcp.flags.ack'] == 'True').astype(int)
df['tcp.flags.fin'] = (df['tcp.flags.fin'] == 'True').astype(int)

In [12]:
# # Convertir les valeurs en chaînes de caractères
# df['ip.id'] = df['ip.id'].astype(str)
# df['ip.dsfield'] = df['ip.dsfield'].astype(str)
# df['tcp.options.sack'] = df['tcp.options.sack'].astype(str)

# # Convertir les valeurs hexadécimales en valeurs décimales
# df['ip.id'] = df['ip.id'].apply(lambda x: int(x, 16))
# df['ip.dsfield'] = df['ip.dsfield'].apply(lambda x: int(x, 16))
# df['tcp.options.sack'] = df['tcp.options.sack'].apply(lambda x: int(x, 16))


In [13]:
number_null_value=df.isnull().sum()
number_null_value

tcp.srcport                     0
tcp.dstport                     0
frame.len                       0
ip.proto                        0
tcp.flags.syn                   0
tcp.flags.ack                   0
tcp.flags.fin                   0
ip.len                          0
tcp.window_size_value           0
tcp.analysis.bytes_in_flight    0
tcp.analysis.push_bytes_sent    0
traficnature                    0
dtype: int64

In [14]:
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]
df

,tcp.srcport,tcp.dstport,frame.len,ip.proto,tcp.flags.syn,tcp.flags.ack,tcp.flags.fin,ip.len,tcp.window_size_value,tcp.analysis.bytes_in_flight,tcp.analysis.push_bytes_sent,traficnature
0,0,0,87,17,0,0,0,73,0,0,0,normal
1,0,0,87,17,0,0,0,73,0,0,0,normal
2,0,0,151,17,0,0,0,137,0,0,0,normal
3,0,0,168,17,0,0,0,154,0,0,0,normal
4,41982,443,74,6,1,0,0,60,64240,0,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...
75470,0,0,74,17,0,0,0,60,0,0,0,normal
75471,0,0,121,17,0,0,0,107,0,0,0,normal
75472,0,0,68,17,0,0,0,54,0,0,0,normal
75473,0,0,82,17,0,0,0,68,0,0,0,normal


# Corrélation entre les différentes caractéristiques

In [15]:
# import seaborn as sns

# # Calculer la matrice de corrélation
# matrix_corr_pearson = df.drop('traficnature', axis=1).corr(method='pearson')

# # Créer une carte de chaleur avec un clustering hiérarchique
# sns.clustermap(matrix_corr_pearson, 
#                xticklabels=True,
#                yticklabels=True,
#                cmap='RdBu_r',
#                linewidth=0.5)


In [16]:
# Calculer la matrice de corrélation absolue
matrix_corr_pearson_abs = df.drop('traficnature', axis=1).corr(method='pearson').abs()

# Créer un masque pour les valeurs supérieures au seuil
mask = np.triu(np.ones_like(matrix_corr_pearson_abs, dtype=bool), k=1)

# Appliquer le masque à la matrice de corrélation
matrix_corr_pearson_masked = matrix_corr_pearson_abs.where(mask)

# Trouver les colonnes qui ont une corrélation supérieure au seuil avec au moins une autre colonne
# to_drop = [column for column in matrix_corr_pearson_masked.columns if any(matrix_corr_pearson_masked[column] > 0.8)]

# # Supprimer les colonnes de df
# df = df.drop(df[to_drop], axis=1)
# print(to_drop)


# Suppression des caractéristiques à haute corrélation

Nous avons supprimés les caractéristiques Bytes et Packets à cause de leur haute corrélation avec les autres caractéristiques restantes
Pour le source Port, nous l'avons supprimés à cause de la présence de beaucoup trop de valeurs (22700 valeurs différentes soit 1/3 de notre dataset) donc nous avons pris encompte que cette caractéristiques n'a aucune importance dans la prédiction d'anomalies

In [17]:
# columnsdrop=['frame.number','tcp.seq','tcp.options.mss_val','tcp.options.wscale.shift']
# df=df.drop(columns=columnsdrop,axis=1)
# df

In [18]:
df.describe()

,tcp.flags.syn,tcp.flags.ack,tcp.flags.fin
count,75475.000000,75475.000000,75475.000000
mean,0.004902,0.136893,0.004478
std,0.069845,0.343737,0.066771
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


In [19]:
df.groupby("traficnature").size()

traficnature
normal    75475
dtype: int64

Nous avons donc 37640 échantillons Allow

# Suppression de la colonne Action

In [20]:
# df=pd.read_csv('AllowedTraffic.csv')
df=df.drop('traficnature', axis=1)
df

,tcp.srcport,tcp.dstport,frame.len,ip.proto,tcp.flags.syn,tcp.flags.ack,tcp.flags.fin,ip.len,tcp.window_size_value,tcp.analysis.bytes_in_flight,tcp.analysis.push_bytes_sent
0,0,0,87,17,0,0,0,73,0,0,0
1,0,0,87,17,0,0,0,73,0,0,0
2,0,0,151,17,0,0,0,137,0,0,0
3,0,0,168,17,0,0,0,154,0,0,0
4,41982,443,74,6,1,0,0,60,64240,0,0
...,...,...,...,...,...,...,...,...,...,...,...
75470,0,0,74,17,0,0,0,60,0,0,0
75471,0,0,121,17,0,0,0,107,0,0,0
75472,0,0,68,17,0,0,0,54,0,0,0
75473,0,0,82,17,0,0,0,68,0,0,0


# Création des deux ensemble de données d'apprentissage et de test

Nous n'avons pas crée d'ensemble de données de validation pour cause de meilleur résultats avec ce scénario (Du à la quantité d'échantillons d'apprentissage qui a augmenté)

In [21]:
from sklearn.model_selection import train_test_split
X=df.values.astype('float64')

# Normalisation des autres caractéristiques en utilisant standardScaler

In [22]:
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

In [23]:

X_train, X_test = train_test_split(X, test_size=0.2,random_state=42)
# X_train, X_val = train_test_split(X_train, test_size=0.25,random_state=42)

In [24]:
X_train

array([[-0.26797498, -0.27691801, -0.0209893 , ..., -0.40191069,
        -0.11289549, -0.11175158],
       [-0.26797498, -0.27691801, -0.0209893 , ..., -0.40191069,
        -0.11289549, -0.11175158],
       [-0.26797498, -0.27691801, -0.0209893 , ..., -0.40191069,
        -0.11289549, -0.11175158],
       ...,
       [-0.26797498, -0.27691801, -0.0209893 , ..., -0.40191069,
        -0.11289549, -0.11175158],
       [-0.23044869,  4.50839875,  1.35158739, ...,  2.49967392,
        15.91768488, 16.6845213 ],
       [-0.26119822,  4.58114186, -0.4299524 , ...,  2.49962875,
        -0.11289549, -0.11175158]])

# Création de notre modéle d'apprentissage

In [25]:
from sklearn import metrics
import numpy as np
import pandas as pd
from IPython.display import display, HTML 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.optimizers import Adam
adam = Adam()
model = Sequential()
model.add(Dense(40 ,input_dim=df.shape[1]))
model.add(Dense(33))
model.add(Dense(27))
model.add(Dense(25))
model.add(Dense(27))
model.add(Dense(df.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer=adam, metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                480       
                                                                 
 dense_1 (Dense)             (None, 33)                1353      
                                                                 
 dense_2 (Dense)             (None, 27)                918       
                                                                 
 dense_3 (Dense)             (None, 25)                700       
                                                                 
 dense_4 (Dense)             (None, 27)                702       
                                                                 
 dense_5 (Dense)             (None, 11)                308       
                                                                 
Total params: 4461 (17.43 KB)
Trainable params: 4461 (17

# Apprentissage de notre modéle

In [26]:
pr=model.fit(X_train,X_train,epochs=100,batch_size=10000)

Epoch 1/100
7/7 [==============================] - 2s 11ms/step - loss: 1.7610 - accuracy: 5.2998e-04
Epoch 2/100
7/7 [==============================] - 0s 14ms/step - loss: 0.8320 - accuracy: 0.7786
Epoch 3/100
7/7 [==============================] - 0s 13ms/step - loss: 0.5062 - accuracy: 0.8760
Epoch 4/100
7/7 [==============================] - 0s 18ms/step - loss: 0.2875 - accuracy: 0.9160
Epoch 5/100
7/7 [==============================] - 0s 17ms/step - loss: 0.1922 - accuracy: 0.9272
Epoch 6/100
7/7 [==============================] - 0s 16ms/step - loss: 0.1415 - accuracy: 0.9467
Epoch 7/100
7/7 [==============================] - 0s 15ms/step - loss: 0.1077 - accuracy: 0.9489
Epoch 8/100
7/7 [==============================] - 0s 16ms/step - loss: 0.0792 - accuracy: 0.9335
Epoch 9/100
7/7 [==============================] - 0s 17ms/step - loss: 0.0611 - accuracy: 0.9645
Epoch 10/100
7/7 [==============================] - 0s 18ms/step - loss: 0.0477 - accuracy: 0.9869
Epoch 11/100
7/

# Utilisation de notre modéle pour prédire le dataset de test et ou tout le dataset Allow ainsi que le dataset de négociations ainsi que des rmse

In [27]:
pred = model.predict(X_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,X_test))
pred = model.predict(X)
score2 = np.sqrt(metrics.mean_squared_error(pred,X))
print(f"Test Data Score (RMSE): {score1}")
print(f"All Data Allowed Score (RMSE): {score2}")

2359/2359 [==============================] - 4s 2ms/step
Test Data Score (RMSE): 0.03185244368393181
All Data Allowed Score (RMSE): 0.033866014518787194


# Comparaison de la prédiction avec le seuil de rmse de toute la data allow

In [28]:
reconstructions = model.predict(X)


2359/2359 [==============================] - 5s 2ms/step


In [34]:
rmse = np.sqrt(np.sum((X - reconstructions)**2, axis=1) / X.shape[1])
allow_mask = rmse < 0.033866014518787194

# Affichage du nombre d'échantillons pris en non Allow

Sachant que X contient 420K,6 échantillons, nous avons 419283 échantillons détécté comme normal

In [35]:
num_true = np.sum(allow_mask)

# Compter le nombre de False
num_false = np.size(allow_mask) - num_true

print(f"Nombre de True : {num_true}")
print(f"Nombre de False : {num_false}")

Nombre de True : 66792
Nombre de False : 8683


# Sauvegarde de notre modéle

In [31]:
model.save('autoencodeur2.h5')

# Sauvegarde de nos normalisateur Countfrequencyencoder et standardScaler

In [32]:
import pickle
with open('allow2.pkl','wb') as f:
    pickle.dump(scaler,f)

In [33]:
# with open('Auto-Encodermodel.pkl', 'wb') as file:
#     pickle.dump(model, file)